In [1]:

basedir = "/home/threedee/repos/Open3D-ML"
datesetbase = "/home/threedee/datasets"

import argparse
import copy
import os
import os.path as osp
import pprint 
import sys
import time
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm
import yaml

os.environ["OPEN3D_ML_ROOT"] = basedir

import open3d.ml as _ml3d

randlanet_semantickitti_cfg = basedir +"/ml3d/configs/randlanet_semantickitti.yml"
randlanet_s3dis_cfg = basedir + "/ml3d/configs/randlanet_s3dis.yml"

randlanet_smartlab_cfg = basedir + "/ml3d/configs/randlanet_smartlab.yml"
kpconv_smartlab_cfg = basedir + "/ml3d/configs/kpconv_smartlab.yml"

ckpt_path = basedir + "/logs/RandLANet_SmartLab_tf/checkpoint/ckpt-14"

kwargs = {
    "framework": "tf",
    "device": "cuda",
    "dataset_path": datesetbase + "/SmartLab",
    "split": "train",
    "ckpt_path": None,
    "cfg_file": kpconv_smartlab_cfg,
}

args = type("args", (object,), kwargs)()

pprint.pprint(kwargs)

Using external Open3D-ML in /home/threedee/repos/Open3D-ML
{'cfg_file': '/home/threedee/repos/Open3D-ML/ml3d/configs/kpconv_smartlab.yml',
 'ckpt_path': None,
 'dataset_path': '/home/threedee/datasets/SmartLab',
 'device': 'cuda',
 'framework': 'tf',
 'split': 'train'}


In [2]:
if args.framework == "torch":
    import open3d.ml.torch as ml3d
else:
    import open3d.ml.tf as ml3d
    import tensorflow as tf

def merge_cfg_file(cfg, args, extra_dict):
    if args.device is not None:
        cfg.pipeline.device = args.device
        cfg.model.device = args.device
    if args.split is not None:
        cfg.pipeline.split = args.split
    if args.dataset_path is not None:
        cfg.dataset.dataset_path = args.dataset_path
    if args.ckpt_path is not None:
        cfg.model.ckpt_path = args.ckpt_path

    return cfg.dataset, cfg.pipeline, cfg.model

device = args.device
gpus = tf.config.experimental.list_physical_devices("GPU")
print(gpus)

if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        if device == "cpu":
            tf.config.set_visible_devices([], "GPU")
        elif device == "cuda":
            tf.config.set_visible_devices(gpus[0], "GPU")
        else:
            idx = device.split(":")[1]
            tf.config.set_visible_devices(gpus[int(idx)], "GPU")
    except RuntimeError as e:
        print(e)

cfg = _ml3d.utils.Config.load_from_file(args.cfg_file)

Pipeline = _ml3d.utils.get_module("pipeline", cfg.pipeline.name, args.framework)
Model = _ml3d.utils.get_module("model", cfg.model.name, args.framework)
Dataset = _ml3d.utils.get_module("dataset", cfg.dataset.name)

cfg_dataset, cfg_pipeline, cfg_model = merge_cfg_file(cfg, args, None)

dataset = Dataset(**cfg_dataset)
model = Model(**cfg_model)
pipeline = Pipeline(model, dataset, **cfg_pipeline)

Unlabeled = [231,87,36]
Floor = [188,169,26]
Wall = [100,244,245]
Robot = [150,30,140]
Human = [0,248,26]
AGV = [18,35,243]

colors = {
    'Unlabeled': [x / 255.0 for x in Unlabeled],
    'Floor': [x / 255.0 for x in Floor],
    'Wall': [x / 255.0 for x in Wall],
    'Robot': [x / 255.0 for x in Robot],
    'Human': [x / 255.0 for x in Human],
    'AGV': [x / 255.0 for x in AGV],
}

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
vis = ml3d.vis.Visualizer()
lut = ml3d.vis.LabelLUT()

pprint.pprint(colors)
i=0
for key, val in colors.items():
    lut.add_label(key, i, val)
    i=i+1

vis.visualize_dataset(dataset, "val", lut, width=2100, height=1600)  # , indices=range(100)

In [3]:
pipeline.cfg_tb = {
    "readme": "readme",
    "cmd_line": "cmd_line",
    "dataset": pprint.pformat(cfg_dataset, indent=2),
    "model": pprint.pformat(cfg_model, indent=2),
    "pipeline": pprint.pformat(cfg_pipeline, indent=2),
}
# print(pipeline.cfg_tb)

if args.split == "test":
    pipeline.run_test()
else:
    pipeline.run_train()

INFO - 2021-03-22 02:11:59,541 - semantic_segmentation - <ml3d.tf.models.kpconv.KPFCNN object at 0x7fc92506f220>
INFO - 2021-03-22 02:11:59,542 - semantic_segmentation - Logging in file : ./logs/KPFCNN_SmartLab_tf/log_train_2021-03-22_02:11:59.txt
INFO - 2021-03-22 02:11:59,544 - smartlab - Found 28 pointclouds for training
INFO - 2021-03-22 02:12:00,702 - smartlab - Found 4 pointclouds for validation
INFO - 2021-03-22 02:12:00,890 - semantic_segmentation - Writing summary in train_log/00002_KPFCNN_SmartLab_tf.
INFO - 2021-03-22 02:12:00,891 - semantic_segmentation - Initializing from scratch.
INFO - 2021-03-22 02:12:00,892 - semantic_segmentation - === EPOCH 0/20 ===
training: 29it [00:14,  2.04it/s]                      
validation: 5it [00:00,  6.71it/s]                       
INFO - 2021-03-22 02:12:15,873 - semantic_segmentation - loss train: 8.401  eval: 9.368
INFO - 2021-03-22 02:12:15,873 - semantic_segmentation - acc train: 0.218  eval: 0.215
INFO - 2021-03-22 02:12:15,874 - s

In [ ]:
# pipeline.load_ckpt(ckpt_path=ckpt_path)
# pipeline.run_test()

# data = test_split.get_data(0)
# attr = test_split.get_attr(0)
# print(attr)

# results = pipeline.run_inference(data)

# pred = (results['predict_labels']).astype(np.int32)
# scores = results['predict_scores']

In [ ]:
test_split = dataset.get_split("test")

vis_points = []
times = []

for idx in range(len(test_split)):

    st = time.perf_counter()
    attr = test_split.get_attr(idx)
    data = test_split.get_data(idx)

    print(attr)
    results = pipeline.run_inference(data)

    pred = (results['predict_labels']).astype(np.int32)

    label = data['label']
    pts = data['point']

    vis_d = {
        "name": attr['name'],
        "points": pts,
        "labels": label,
        "pred": pred,
    }

    vis_points.append(vis_d)
    et = time.perf_counter()
    times.append(et-st)

print("\n")
print(times)
print(f"Average time {np.mean(times):0.4f} seconds")

In [ ]:
v = ml3d.vis.Visualizer()
lut = ml3d.vis.LabelLUT()

pprint.pprint(colors)

label_names = dataset.get_label_to_names()
pprint.pprint(label_names)

for (c, cv), (l, lv) in zip(colors.items(), label_names.items()):
    lut.add_label(lv, l, cv)

v.set_lut("labels", lut)
v.set_lut("pred", lut)

v.visualize(vis_points)

In [ ]:
test_split = dataset.get_split("test")

for idx in range(len(test_split)):
    attr = test_split.get_attr(idx)
    data = test_split.get_data(idx)
    
    unique, counts = np.unique(data['label'], return_counts=True)
    print(np.asarray((unique, counts)))